# User scraping 

- 강남역 맛집 리뷰 수가 많은 유저의 myplace를 크롤링

### 문제

1. 인기 리뷰어라면 핫플레이스만 다녀서 평범한 음식점에 대한 정보는 못얻지 않을까?

In [1]:
# 웹 드라이버 설정
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager 

# 대기 관련 라이브러리
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 

# 예외 처리 관련 라이브러리
from selenium.common.exceptions import TimeoutException, NoSuchElementException  

# 웹 요소 찾기 관련 라이브러리
from selenium.webdriver.common.by import By  
from selenium.webdriver.support.ui import Select  
from selenium.webdriver.common.keys import Keys  

# 그 외 
import time 
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup 
import numpy as np  
import pandas as pd 
import re  
from tqdm import tqdm  # 반복문 진행 상황 시각화 모듈
import zipfile
import os

c:\Users\hare0\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


https://m.place.naver.com/my/5c36b9f1e511a8856c50c832/review?v=2
https://m.place.naver.com/my/5e1370ce8f87a842bc017bc5/review?v=2
https://m.place.naver.com/my/5bd2f3682db9d1bdd38eedca/review?v=2
https://m.place.naver.com/my/5f1dd9049ec8258e4a657f78/review?v=2
https://m.place.naver.com/my/review/65bda752011aacbd86f5ef9e?v=2

In [2]:
# webdriver_manager를 사용하여 ChromeDriver 다운로드 및 설정
driver = webdriver.Chrome(ChromeDriverManager().install())

# 주소 이동 (SUNsDay 나 아)
url = 'https://m.place.naver.com/my/5c2b07cbc98dc4c88b3ac001/review?v=2'
driver.get(url)
time.sleep(1)


In [4]:
# 저장 
User_df = pd.DataFrame()
time.sleep(3)  

# 맨 처음 게시물 클릭
driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div[1]/div[5]/div/ul/li[1]/button').click()



finish
rudtjswogud / 맛있어요.. 트러플향도 좋고 안주도 맛있고 특히 리치하이볼(양귀비) 제취향.내용 더보기 / 2.3.토 / 1번째 방문
gks**** / 웰컴드링크도 나오는 수컷웅!
하이볼도 다양하고 음식도 맛있습니다
데이트 장소로도 추천해요~! / 1.31.수 / 1번째 방문
먹꾼 / 게내장파스타 안비리고 매콤해서 정말 맛있었어요 👍👍 양귀비하이볼도 많이 달지 않고 상큼하게 맛있어서 좋았고 삼겹살도 부추랑 와서비흘 곁들여먹으니 완전 존맛 👍👍 그런데 예약해야 테이블석에 앉을 수 있더라고요 ~~ 편한 자리에서 식사하고 싶다면 예약 추천합니다 ~~ / 1.31.수 / 1번째 방문
쏠라도레미 / 맛있어요~!
왕십리 술집 중 분위기 좋습니다 / 1.27.토 / 1번째 방문
typerin08 / 분위기도 좋고 편하게 먹기 좋아요! 잘 먹구 갑니다! / 1.27.토 / 1번째 방문
시키렁키 / 하이볼종류가 정말 다양하고 전문적인게 딱 느껴지네요.
주당으로서 여기 진짜 사랑합니다. / 1.27.토 / 1번째 방문
연두해요21 / 너무 맛있고 따뜻한 음식이라서 좋았어요!!!완전 왕십리 맛집 !!!!!!다음에도 올거 같아요!체고!!❤️ / 1.26.금 / 1번째 방문
son**** / 웰컴 드링크도 센스있고 좋았어요! 고급스러운 분위기에 다양한 칵테일을 즐길 수 있어서 좋았습니다! 인테리어가 예뻐서 데이트하기도 좋을것 같아요! / 1.26.금 / 1번째 방문
예 그리나 /  / 1.24.수 / 1번째 방문
gml**** / 친절하시고 간단히 이차하기에 너무 좋아요 보조배터리도 대여되서 너무 좋은거같아요!! / 1.24.수 / 1번째 방문
e ni316 /  / 1.23.화 / 1번째 방문
rkd**** / 기본안주랑 웰컴하이볼도 맛있어서 기분좋게 저녁식사하고 가요! 분위기도 적당히 어둡고 선곡도 분위기 있어서 데이트 코스로 딱이에요. 실제로 옆자리 소개팅 하시는 거 같았어요ㅎㅎ 게장파스타는 꼭 시키세요ㅜ / 1.23.화 / 1번째 방문
aje**** / 